In [6]:
import librosa
from os import listdir
from os import path
from os.path import isfile, join
import os
import numpy as np

In [2]:
def cal_mean(dir_in,nfft,hop_size,wlen):
    for f in listdir(dir_in):
        sum_col = 0
        new_path = join(dir_in,f)
        x,fs = librosa.load(new_path,sr=None)
                
        reverb_stft = librosa.stft(x,n_fft = nfft,hop_length = hop_size,win_length = wlen)
        dims = reverb_stft.shape
        row_n = dims[0]
        col_n = dims[1]

        new_comb = np.zeros((dim_size*row_n,col_n))# create a numpy array of size [dim_size * row_n,col_n]
        sum_m = np.zeros((dim_size*row_n,))
        # This way can make sure three features can pass in right time sequence
        phase = np.angle(reverb_stft)
        cos_stft = np.cos(phase)
        sin_stft = np.sin(phase)
        logmag_stft = np.log(np.absolute(reverb_stft)+np.finfo(float).eps)
        new_comb[0:row_n,:] = logmag_stft
        new_comb[row_n:2*row_n,:] = sin_stft
        new_comb[2*row_n:,:] = cos_stft
        sum_m += np.sum(new_comb,axis=1)
        sum_col += col_n
            
    mean_comb = sum_m/col_n
    return mean_comb
                

In [3]:
def cal_var(dir_in,mean_c,nfft,hop_size,wlen):
    for f in listdir(dir_in):
        sum_col = 0
        new_path = join(dir_in,f)
        x,fs = librosa.load(new_path,sr=None)
                
        reverb_stft = librosa.stft(x,n_fft = nfft,hop_length = hop_size,win_length = wlen)
        dims = reverb_stft.shape
        row_n = dims[0]
        col_n = dims[1]

        new_comb = np.zeros((dim_size*row_n,col_n))# create a numpy array of size [dim_size * row_n,col_n]
        sum_m = np.zeros((dim_size*row_n,))
        # This way can make sure three features can pass in right time sequence
        phase = np.angle(reverb_stft)
        cos_stft = np.cos(phase)
        sin_stft = np.sin(phase)
        logmag_stft = np.log(np.absolute(reverb_stft)+np.finfo(float).eps)
        new_comb[0:row_n,:] = logmag_stft
        new_comb[row_n:2*row_n,:] = sin_stft
        new_comb[2*row_n:,:] = cos_stft
        new_input = np.square((new_comb.transpose()-mean_c).transpose())
        #print(new_input.shape)
        sum_m += np.sum(new_input,axis=1)
        sum_col += col_n
        
    var_sq = sum_m / sum_col
    var = np.sqrt(var_sq)
    return var
                

In [4]:
Fs = 8e3
nfft = 2 ** 7 # recomended to be power of 2 and greater than win_len, this is the smallest that meet 10 millisec
wlen_time = [10,20,40,60,80] # millisec
overlap_perc = [0.125,0.25,0.5,0.75] # percentage of win_len

t60s = [0.3,0.6,0.9]
num_samples = 5000
num_rirs = 500
dim_size = 3

In [5]:
trainPath = '/data/liyuy/PROJECTS/DEREVERB3/timit_8k/reverb/train/'
outPath = '/data/liyuy/PROJECTS/DEREVERB3/timit_8k/reverb_train/'
for a in range(0,len(wlen_time)):
    wlen = int(wlen_time[a] * 1e-3 * Fs)
    if wlen > nfft: # To meet the condition, we multiply by 2 when nfft < wlen
        nfft = nfft * 2
    for b in range(0,len(overlap_perc)):
        overlap = int(overlap_perc[b] * wlen)
        hop_size = wlen - overlap
        newout_path = outPath + 'train_wlen_' + '%d' % wlen + '_nfft_' + '%d' % nfft + '_overlap_' + '%d' % overlap + '/'
        if path.exists(newout_path) == False:
            os.mkdir(newout_path)
        roomNum = 1
        i_loc = 1
        mean_comb = cal_mean(trainPath,nfft,hop_size,wlen)
        var = cal_var(trainPath,mean_comb,nfft,hop_size,wlen)
        for i in range(1,num_samples+1):
            if i != 1 and (i - 1) % num_rirs == 0:
                roomNum += 1
                i_loc = 1
            for x in range(0,len(t60s)):
                t60 = t60s[x]

                input_path = trainPath + 'reverb_rir' + "{0:0=4d}".format(i) + '_roomNum' + '%d' % roomNum + '_t60' + '%.1f' % t60 + '_loc' + '%d' % i_loc + '_8kHz.wav'

                x,fs = librosa.load(input_path,sr=None)
                
                reverb_stft = librosa.stft(x,n_fft = nfft,hop_length = hop_size,win_length = wlen)
                dims = reverb_stft.shape
                row_n = dims[0]
                col_n = dims[1]
                
                new_comb = np.zeros((dim_size*row_n,col_n)) # create a numpy array of size [dim_size * row_n,col_n]
                # This way can make sure three features can pass in right time sequence
                phase = np.angle(reverb_stft)
                cos_stft = np.cos(phase)
                sin_stft = np.sin(phase)
                logmag_stft = np.log(np.absolute(reverb_stft)+np.finfo(float).eps)
                new_comb[0:row_n,:] = logmag_stft
                new_comb[row_n:2*row_n,:] = sin_stft
                new_comb[2*row_n:,:] = cos_stft
                out_comb = np.divide((new_comb.transpose() - mean_comb),(var+np.finfo(float).eps)).transpose()
                print(out_comb)
                #output_path = newout_path + 'reverb_rir' + "{0:0=4d}".format(i) + '_roomNum' + '%d' % roomNum + '_t60' + '%.1f' % t60 + '_loc' + '%d' % i_loc + '_'+'%d'%(Fs/1000) +'kHz.npy'
                #np.save(output_path,out_comb)
            i_loc += 1
    
        

[[ 1.17845768  1.10714736  1.16612692 ... -0.82443078 -0.82443078
  -0.82443078]
 [ 1.15646221  1.11886271  1.12757521 ... -0.82416184 -0.82416184
  -0.82416184]
 [ 1.11436624  1.13502161  0.99403446 ... -0.82255096 -0.82255096
  -0.82255096]
 ...
 [ 0.61686214  0.57182821 -2.28538129 ...  0.61686214  0.61686214
   0.61686214]
 [-2.38075236 -2.36768949  0.46795464 ...  0.57408088  0.57408088
   0.57408088]
 [ 0.51316014  0.51316014 -1.94870941 ...  0.51316014  0.51316014
   0.51316014]]
[[ 1.17619052  1.0152215   1.14135035 ... -0.82443078 -0.82443078
  -0.82443078]
 [ 1.15567303  1.11429778  1.09795347 ... -0.82416184 -0.82416184
  -0.82416184]
 [ 1.11686629  1.12891442  0.98694177 ... -0.82255096 -0.82255096
  -0.82255096]
 ...
 [ 0.61686214  0.30142493 -2.24832525 ...  0.61686214  0.61686214
   0.61686214]
 [-2.38075236  0.01711193  0.57381484 ...  0.57408088  0.57408088
   0.57408088]
 [ 0.51316014 -1.94870941 -1.94870941 ...  0.51316014  0.51316014
   0.51316014]]
[[ 1.17619052  1

[[ 1.11520871  1.20399208  1.12375115 ... -0.82443078 -0.82443078
  -0.82443078]
 [ 1.13117063  1.18997866  1.168076   ... -0.82416184 -0.82416184
  -0.82416184]
 [ 1.13821989  1.17206681  1.18116926 ... -0.82255096 -0.82255096
  -0.82255096]
 ...
 [ 0.61686214  0.60871935  0.39128505 ...  0.61686214  0.61686214
   0.61686214]
 [-2.38075236 -2.11135463 -2.29006567 ...  0.57408088  0.57408088
   0.57408088]
 [ 0.51316014  0.51316014  0.51316014 ...  0.51316014  0.51316014
   0.51316014]]
[[ 1.17665903  1.17732852  1.1794604  ... -0.82443078 -0.82443078
  -0.82443078]
 [ 1.14512453  1.14072909  1.14819964 ... -0.82416184 -0.82416184
  -0.82416184]
 [ 1.08825505  1.04371329  1.08996145 ... -0.82255096 -0.82255096
  -0.82255096]
 ...
 [ 0.61686214 -0.96420288 -2.25902039 ...  0.61686214  0.61686214
   0.61686214]
 [-2.38075236 -1.70264447  0.52706745 ...  0.57408088  0.57408088
   0.57408088]
 [ 0.51316014  0.51316014 -1.94870941 ...  0.51316014  0.51316014
   0.51316014]]
[[ 1.17665903  1

[[ 1.15016441  1.16933477  1.15009311 ... -0.82443078 -0.82443078
  -0.82443078]
 [ 1.12634679  1.12386017  1.11969762 ... -0.82416184 -0.82416184
  -0.82416184]
 [ 1.09395391  1.1001315   1.10984876 ... -0.82255096 -0.82255096
  -0.82255096]
 ...
 [-2.32324371  0.61379905  0.28472734 ...  0.61686214  0.61686214
   0.61686214]
 [ 0.57408088 -2.34187391 -1.07038946 ...  0.57408088  0.57408088
   0.57408088]
 [-1.94870941  0.51316014 -1.94870941 ...  0.51316014  0.51316014
   0.51316014]]
[[ 1.13786215  1.17443956  1.1393766  ... -0.82443078 -0.82443078
  -0.82443078]
 [ 1.12956906  1.14175052  1.09190511 ... -0.82416184 -0.82416184
  -0.82416184]
 [ 1.11902726  1.0783766   1.0354549  ... -0.82255096 -0.82255096
  -0.82255096]
 ...
 [-2.32324371 -1.30146585  0.41973371 ...  0.61686214  0.61686214
   0.61686214]
 [ 0.57408088  0.08807792 -2.3561662  ...  0.57408088  0.57408088
   0.57408088]
 [-1.94870941 -1.94870941  0.51316014 ...  0.51316014  0.51316014
   0.51316014]]
[[ 1.13786215  1

[[ 1.17202268  1.14610016  1.19006852 ... -0.82443078 -0.82443078
  -0.82443078]
 [ 1.14378889  1.11517714  1.15626983 ... -0.82416184 -0.82416184
  -0.82416184]
 [ 1.08901825  1.07660614  1.11249501 ... -0.82255096 -0.82255096
  -0.82255096]
 ...
 [ 0.61686214 -1.15355368  0.01620671 ...  0.61686214  0.61686214
   0.61686214]
 [-2.38075236 -0.30452832 -2.16233035 ...  0.57408088  0.57408088
   0.57408088]
 [ 0.51316014 -1.94870941  0.51316014 ...  0.51316014  0.51316014
   0.51316014]]
[[ 1.17202268  1.14671518  1.18960338 ... -0.82443078 -0.82443078
  -0.82443078]
 [ 1.14378889  1.11593843  1.15515372 ... -0.82416184 -0.82416184
  -0.82416184]
 [ 1.08901825  1.07817204  1.1151974  ... -0.82255096 -0.82255096
  -0.82255096]
 ...
 [ 0.61686214 -1.28572439  0.41257333 ...  0.61686214  0.61686214
   0.61686214]
 [-2.38075236 -1.03600201 -2.33001299 ...  0.57408088  0.57408088
   0.57408088]
 [ 0.51316014  0.51316014  0.51316014 ...  0.51316014  0.51316014
   0.51316014]]
[[ 1.17202268  1

[[ 1.16601689  1.1106183   1.14692859 ... -0.82443078 -0.82443078
  -0.82443078]
 [ 1.14013672  1.06684572  1.0890856  ... -0.82416184 -0.82416184
  -0.82416184]
 [ 1.09315514  1.07619814  1.08303991 ... -0.82255096 -0.82255096
  -0.82255096]
 ...
 [-2.32324371 -2.3102994  -0.54457146 ...  0.61686214  0.61686214
   0.61686214]
 [ 0.57408088  0.5686661  -0.19337716 ...  0.57408088  0.57408088
   0.57408088]
 [-1.94870941 -1.94870941 -1.94870941 ...  0.51316014  0.51316014
   0.51316014]]
[[ 1.16601689  1.11591934  1.14206354 ... -0.82443078 -0.82443078
  -0.82443078]
 [ 1.14013672  1.06452096  1.08259707 ... -0.82416184 -0.82416184
  -0.82416184]
 [ 1.09315514  1.07597689  1.07977888 ... -0.82255096 -0.82255096
  -0.82255096]
 ...
 [-2.32324371 -2.32228924 -0.68278223 ...  0.61686214  0.61686214
   0.61686214]
 [ 0.57408088  0.57295132  0.12201509 ...  0.57408088  0.57408088
   0.57408088]
 [-1.94870941 -1.94870941 -1.94870941 ...  0.51316014  0.51316014
   0.51316014]]
[[ 1.1164941   1

...

   0.51316014]]
[[ 1.15728881  1.12276513  1.16367725 ... -0.82443078 -0.82443078
  -0.82443078]
 [ 1.15119187  1.08986668  1.12793082 ... -0.82416184 -0.82416184
  -0.82416184]
 [ 1.14038039  1.04381841  1.0495981  ... -0.82255096 -0.82255096
  -0.82255096]
 ...
 [ 0.61686214 -0.20794763 -2.19995502 ...  0.61686214  0.61686214
   0.61686214]
 [ 0.57408088 -1.9084559   0.51929582 ...  0.57408088  0.57408088
   0.57408088]
 [-1.94870941  0.51316014 -1.94870941 ...  0.51316014  0.51316014
   0.51316014]]
[[ 1.16577451  1.16380414  1.16368987 ... -0.82443078 -0.82443078
  -0.82443078]
 [ 1.14640877  1.12497186  1.1233361  ... -0.82416184 -0.82416184
  -0.82416184]
 [ 1.11448162  1.04440974  1.00569817 ... -0.82255096 -0.82255096
  -0.82255096]
 ...
 [-2.32324371 -2.20130545  0.1120818  ...  0.61686214  0.61686214
   0.61686214]
 [ 0.57408088  0.04639465 -1.67280892 ...  0.57408088  0.57408088
   0.57408088]
 [-1.94870941  0.51316014  0.51316014 ...  0.51316014  0.51316014
   0.51316014]]


   0.51316014]]
[[ 1.19744431  1.16795922  1.17341859 ... -0.82443078 -0.82443078
  -0.82443078]
 [ 1.15243217  1.14253073  1.14460303 ... -0.82416184 -0.82416184
  -0.82416184]
 [ 0.99459867  1.07732317  1.12311751 ... -0.82255096 -0.82255096
  -0.82255096]
 ...
 [ 0.61686214  0.20143677 -2.28442498 ...  0.61686214  0.61686214
   0.61686214]
 [-2.38075236 -2.17957091  0.57407867 ...  0.57408088  0.57408088
   0.57408088]
 [ 0.51316014  0.51316014 -1.94870941 ...  0.51316014  0.51316014
   0.51316014]]
[[ 1.19744431  1.16795922  1.16336013 ... -0.82443078 -0.82443078
  -0.82443078]
 [ 1.15243217  1.14253073  1.14016278 ... -0.82416184 -0.82416184
  -0.82416184]
 [ 0.99459867  1.07732317  1.13112546 ... -0.82255096 -0.82255096
  -0.82255096]
 ...
 [ 0.61686214  0.20143677 -2.31778065 ...  0.61686214  0.61686214
   0.61686214]
 [-2.38075236 -2.17957091  0.56160775 ...  0.57408088  0.57408088
   0.57408088]
 [ 0.51316014  0.51316014 -1.94870941 ...  0.51316014  0.51316014
   0.51316014]]


[[ 1.17358406  1.17343455  1.18044495 ... -0.82443078 -0.82443078
  -0.82443078]
 [ 1.14187893  1.13887787  1.145304   ... -0.82416184 -0.82416184
  -0.82416184]
 [ 1.08174452  1.07499924  1.07816051 ... -0.82255096 -0.82255096
  -0.82255096]
 ...
 [-2.32324371 -2.00276175 -1.07928514 ...  0.61686214  0.61686214
   0.61686214]
 [ 0.57408088  0.35017496 -0.44738766 ...  0.57408088  0.57408088
   0.57408088]
 [-1.94870941 -1.94870941 -1.94870941 ...  0.51316014  0.51316014
   0.51316014]]
[[ 1.17358406  1.17343455  1.18044495 ... -0.82443078 -0.82443078
  -0.82443078]
 [ 1.14187893  1.13887787  1.145304   ... -0.82416184 -0.82416184
  -0.82416184]
 [ 1.08174452  1.07499924  1.07816051 ... -0.82255096 -0.82255096
  -0.82255096]
 ...
 [-2.32324371 -2.00276175 -1.07928514 ...  0.61686214  0.61686214
   0.61686214]
 [ 0.57408088  0.35017496 -0.44738766 ...  0.57408088  0.57408088
   0.57408088]
 [-1.94870941 -1.94870941 -1.94870941 ...  0.51316014  0.51316014
   0.51316014]]
[[ 1.19342649  1

[[ 1.17656275  1.19467238  1.18412078 ... -0.82443078 -0.82443078
  -0.82443078]
 [ 1.14120498  1.16237263  1.1475823  ... -0.82416184 -0.82416184
  -0.82416184]
 [ 1.06645462  1.10242589  1.06077973 ... -0.82255096 -0.82255096
  -0.82255096]
 ...
 [ 0.61686214 -1.43382534 -0.01228174 ...  0.61686214  0.61686214
   0.61686214]
 [-2.38075236 -0.74553656 -2.35317706 ...  0.57408088  0.57408088
   0.57408088]
 [ 0.51316014  0.51316014  0.51316014 ...  0.51316014  0.51316014
   0.51316014]]
[[ 1.18241058  1.18116195  1.18511655 ... -0.82443078 -0.82443078
  -0.82443078]
 [ 1.14366037  1.15159842  1.15251653 ... -0.82416184 -0.82416184
  -0.82416184]
 [ 1.04448703  1.11177255  1.10215937 ... -0.82255096 -0.82255096
  -0.82255096]
 ...
 [-2.32324371 -2.32323714  0.18947147 ...  0.61686214  0.61686214
   0.61686214]
 [ 0.57408088  0.57395618 -2.37975251 ...  0.57408088  0.57408088
   0.57408088]
 [-1.94870941 -1.94870941  0.51316014 ...  0.51316014  0.51316014
   0.51316014]]
[[ 1.18241058  1

KeyboardInterrupt: 

In [6]:
Fs = 8e3
nfft = 2 ** 7
wlen_time = [10,20,40,60,80]
overlap_perc = [0.125,0.25,0.5,0.75]

t60s = [0.3,0.6,0.9]
num_samples = 500
num_rirs = 50
dim_size = 3

In [12]:
validPath = '/data/liyuy/PROJECTS/DEREVERB3/timit_8k/reverb/valid/'
outPath = '/data/liyuy/PROJECTS/DEREVERB3/timit_8k/reverb_valid/'
for a in range(0,len(wlen_time)):
    wlen = int(wlen_time[a] * 1e-3 * Fs)
    if wlen > nfft:
        nfft = nfft * 2
    for b in range(0,len(overlap_perc)):
        overlap = int(overlap_perc[b] * wlen)
        hop_size = wlen - overlap
        newout_path = outPath + 'valid_wlen_' + '%d' % wlen + '_nfft_' + '%d' % nfft + '_overlap_' + '%d' % overlap + '/'
        if path.exists(newout_path) == False:
            os.mkdir(newout_path)
        roomNum = 1
        i_loc = 1
        mean_comb = cal_mean(trainPath,nfft,hop_size,wlen)
        var = cal_var(trainPath,mean_comb,nfft,hop_size,wlen)
        for i in range(1,num_samples+1):
            if i != 1 and (i - 1) % num_rirs == 0:
                roomNum += 1

            for x in range(0,len(t60s)):
                t60 = t60s[x]

                input_path = validPath + 'reverb_rir' + "{0:0=4d}".format(i) + '_roomNum' + '%d' % roomNum + '_t60' + '%.1f' % t60 + '_loc' + '%d' % i_loc + '_8kHz.wav'

                x,fs = librosa.load(input_path,sr=None)
                
                reverb_stft = librosa.stft(x,n_fft = nfft,hop_length = hop_size,win_length = wlen)
                dims = reverb_stft.shape
                
                row_n = dims[0]
                col_n = dims[1]
                
                new_comb = np.zeros((dim_size*row_n,col_n))
                phase = np.angle(reverb_stft)
                cos_stft = np.cos(phase)
                sin_stft = np.sin(phase)
                logmag_stft = np.log(np.absolute(reverb_stft)+np.finfo(float).eps)
                new_comb[0:row_n,:] = logmag_stft
                new_comb[row_n:2*row_n,:] = sin_stft
                new_comb[2*row_n:,:] = cos_stft
                out_comb = np.divide((new_comb.transpose() - mean_comb),(var+np.finfo(float).eps)).transpose()
                output_path = newout_path + 'reverb_rir' + "{0:0=4d}".format(i) + '_roomNum' + '%d' % roomNum + '_t60' + '%.1f' % t60 + '_loc' + '%d' % i_loc + '_8kHz.npy'
                np.save(output_path,out_comb)
            i_loc += 1
    
        

52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300
52999
(65, 5300)
65 5300


KeyboardInterrupt: 

In [7]:
Fs = 8e3
nfft = 2 ** 7 # recomended to be power of 2 and greater than win_len, this is the smallest that meet 10 millisec
wlen_time = [10,20,40,60,80] # millisec
overlap_perc = [0.125,0.25,0.5,0.75] # percentage of win_len

t60s = [0.3,0.6,0.9]
num_samples = 500
num_rirs = 500
dim_size = 3
roomNums = [11,12,13,14] # 4 rooms with unseen RIRs

In [8]:
testPath = '/data/liyuy/PROJECTS/DEREVERB3/timit_8k/reverb/test'
outPath = '/data/liyuy/PROJECTS/DEREVERB3/timit_8k/reverb_test'
for r in range(0,len(roomNums)):
    roomNum = roomNums[r]
    new_test = testPath + '%d'%(r+1) + '/'
    new_out = outPath + '%d'%(r+1) + '/'
    if path.exists(new_out) == False:
        os.mkdir(new_out)
    for a in range(0,len(wlen_time)):
        wlen = int(wlen_time[a] * 1e-3 * Fs)
        if wlen > nfft:
            nfft = nfft * 2
        for b in range(0,len(overlap_perc)):
            overlap = int(overlap_perc[b] * wlen)
            hop_size = wlen - overlap
            newout_path = new_out + 'test_wlen_' + '%d' % wlen + '_nfft_' + '%d' % nfft + '_overlap_' + '%d' % overlap + '/'
            if path.exists(newout_path) == False:
                os.mkdir(newout_path)
            
            i_loc = 1
            mean_comb = cal_mean(trainPath,nfft,hop_size,wlen)
            var = cal_var(trainPath,mean_comb,nfft,hop_size,wlen)
            for i in range(1,num_samples+1):

                for x in range(0,len(t60s)):
                    t60 = t60s[x]

                    input_path = new_test + 'reverb_rir' + "{0:0=4d}".format(i) + '_roomNum' + '%d' % roomNum + '_t60' + '%.1f' % t60 + '_loc' + '%d' % i_loc + '_8kHz.wav'

                    x,fs = librosa.load(input_path,sr=None)
                    reverb_stft = librosa.stft(x,n_fft = nfft,hop_length = hop_size,win_length = wlen)
                    dims = reverb_stft.shape
                    row_n = dims[0]
                    col_n = dims[1]
                    new_comb = np.zeros((dim_size*row_n,col_n))
                    phase = np.angle(reverb_stft)
                    cos_stft = np.cos(phase)
                    sin_stft = np.sin(phase)
                    logmag_stft = np.log(np.absolute(reverb_stft)+np.finfo(float).eps)
                    new_comb[0:row_n,:] = logmag_stft
                    new_comb[row_n:2*row_n,:] = sin_stft
                    new_comb[2*row_n:,:] = cos_stft
                    out_comb = np.divide((new_comb.transpose() - mean_comb),(var+np.finfo(float).eps)).transpose()
                    output_path = newout_path + 'reverb_rir' + "{0:0=4d}".format(i) + '_roomNum' + '%d' % roomNum + '_t60' + '%.1f' % t60 + '_loc' + '%d' % i_loc + '_8kHz.npy'
                    np.save(output_path,out_comb)
                i_loc += 1



In [2]:
Fs = 8e3
nfft = 2 ** 7 # recomended to be power of 2 and greater than win_len, this is the smallest that meet 10 millisec
wlen_time = [10,20,40,60,80] # millisec
overlap_perc = [0.125,0.25,0.5,0.75] # percentage of win_len

t60s = [0.3,0.6,0.9]
num_samples = 500
num_rirs = 50
dim_size = 3
roomNums = 1 # start room number, total 10 rooms (seen RIRs)

In [4]:
testPath = '/data/liyuy/PROJECTS/DEREVERB3/timit_8k/reverb/test5/'
outPath = '/data/liyuy/PROJECTS/DEREVERB3/timit_8k/reverb_test5/'
if path.exists(outPath) == False:
            os.mkdir(outPath)
for a in range(0,len(wlen_time)):
    wlen = int(wlen_time[a] * 1e-3 * Fs)
    if wlen > nfft:
        nfft = nfft * 2
    for b in range(0,len(overlap_perc)):
        overlap = int(overlap_perc[b] * wlen)
        hop_size = wlen - overlap
        newout_path = outPath + 'test_wlen_' + '%d' % wlen + '_nfft_' + '%d' % nfft + '_overlap_' + '%d' % overlap + '/'
        if path.exists(newout_path) == False:
            os.mkdir(newout_path)
        roomNum = 1
        i_loc = 1
        mean_comb = cal_mean(trainPath,nfft,hop_size,wlen)
        var = cal_var(trainPath,mean_comb,nfft,hop_size,wlen)
        for i in range(1,num_samples+1):
            if i != 1 and (i - 1) % num_rirs == 0:
                roomNum += 1

            for x in range(0,len(t60s)):
                t60 = t60s[x]

                input_path = testPath + 'reverb_rir' + "{0:0=4d}".format(i) + '_roomNum' + '%d' % roomNum + '_t60' + '%.1f' % t60 + '_loc' + '%d' % i_loc + '_8kHz.wav'

                x,fs = librosa.load(input_path,sr=None)
                reverb_stft = librosa.stft(x,n_fft = nfft,hop_length = hop_size,win_length = wlen)
                dims = reverb_stft.shape
                row_n = dims[0]
                col_n = dims[1]
                new_comb = np.zeros((dim_size*row_n,col_n))
                phase = np.angle(reverb_stft)
                cos_stft = np.cos(phase)
                sin_stft = np.sin(phase)
                logmag_stft = np.log(np.absolute(reverb_stft)+np.finfo(float).eps)
                new_comb[0:row_n,:] = logmag_stft
                new_comb[row_n:2*row_n,:] = sin_stft
                new_comb[2*row_n:,:] = cos_stft
                out_comb = np.divide((new_comb.transpose() - mean_comb),(var+np.finfo(float).eps)).transpose()
                output_path = newout_path + 'reverb_rir' + "{0:0=4d}".format(i) + '_roomNum' + '%d' % roomNum + '_t60' + '%.1f' % t60 + '_loc' + '%d' % i_loc + '_8kHz.npy'
                np.save(output_path,out_comb)
            i_loc += 1
    
        

In [24]:
a = np.matrix('1 2 3; 3 5 4')
b = np.array([1,2])
c = np.divide(a.transpose(),b).transpose()
d = (a.transpose() - b).transpose()
print(d)

[[0 1 2]
 [1 3 2]]
